In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from adjustText import adjust_text

from compEL.iv import discrete_iv
from plot.plot_utilities import hist1d, hist2d, edgeformat
from database.query import get_trajid, get_trajattr

In [ ]:
%store -r traj_ids
voltages = np.sort(np.unique(get_trajattr(traj_ids, 'voltage').values))

%store -r e1124sc
%store -r e1126sc
%store -r r334sc
%store -r states_df

metrics_df = states_df.copy()
metrics_df = pd.merge(metrics_df, r334sc[['traj_id', 'timestep', 'x', 'y', 'z']], on=['traj_id', 'timestep'])
metrics_df.rename(columns={'x': 'sc_x_334', 'y': 'sc_y_334', 'z': 'sc_z_334'}, inplace=True)
metrics_df = pd.merge(metrics_df, e1124sc[['traj_id', 'timestep', 'x', 'y', 'z']], on=['traj_id', 'timestep'])
metrics_df.rename(columns={'x': 'sc_x_1124', 'y': 'sc_y_1124', 'z': 'sc_z_1124'}, inplace=True)
metrics_df = pd.merge(metrics_df, e1126sc[['traj_id', 'timestep', 'x', 'y', 'z']], on=['traj_id', 'timestep'])
metrics_df.rename(columns={'x': 'sc_x_1126', 'y': 'sc_y_1126', 'z': 'sc_z_1126'}, inplace=True)

metrics_df = metrics_df.query("tmpc1v2 == 3")

# Salt bridge between R334 and ECL-6 GLU

In [ ]:
# Distance between sidechain centers
d334_1124 = metrics_df[['sc_x_334', 'sc_y_334', 'sc_z_334']].values - metrics_df[['sc_x_1124', 'sc_y_1124', 'sc_z_1124']].values
d334_1124 = np.linalg.norm(d334_1124, axis=1)
metrics_df['d334_1124'] = d334_1124

d334_1126 = metrics_df[['sc_x_334', 'sc_y_334', 'sc_z_334']].values - metrics_df[['sc_x_1126', 'sc_y_1126', 'sc_z_1126']].values
d334_1126 = np.linalg.norm(d334_1126, axis=1)
metrics_df['d334_1126'] = d334_1126

In [ ]:
fig, axs = plt.subplots()

hist2d(d334_1124, d334_1126, bins=50, range=[[0,30],[0,30]]).dens2d_preset2(axs, lw=0.3, nlevels=15, no_last_contour_line=False)

axs.set_xlabel('d334_1124')
axs.set_ylabel('d334_1126')

In [ ]:
# Versus GLU states
fig, axs1 = plt.subplots(2, 1, sharex=True)
for ax in axs1:
    edgeformat(ax)

fig, axs = plt.subplots(1, 2, figsize=(8,4))

loop_dict = {0: "loop-out", 1: "loop-in"}

for s in [0,1]:
    sidechain_select = (metrics_df['e1126'] == s).values

    hist2d(d334_1124[sidechain_select], d334_1126[sidechain_select], bins=30, range=[[0,30],[0,30]]).dens2d_preset2(axs[s], lw=0.3, nlevels=15, no_last_contour_line=False)
    hist_minor = hist1d(d334_1124[sidechain_select], bins=60, range=[0,30])
    hist_minor.plot(axs1[0], lw=2, label=f"{loop_dict[s]}; %salt-bridge = {round(hist_minor.integral_dens(0,5.5),2)}")
    
    hist_major = hist1d(d334_1126[sidechain_select], bins=60, range=[0,30])
    hist_major.plot(axs1[1], lw=2, label=f"{loop_dict[s]}; %salt-bridge = {round(hist_major.integral_dens(0,5.5),2)}")

    axs[s].set_xlabel('d334_1124')
    axs[s].set_ylabel('d334_1126')

for ax in axs1:
    ax.legend()
    # ax.set_xlim(0,10)

# R334 vs ECL-6 glu states

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(10,6), height_ratios=[1,2], width_ratios=[2,2,1], gridspec_kw={'wspace': 0, 'hspace': 0}, sharex='col', sharey='row')

# fig, axs0 = plt.subplots()
# axs0.grid(True)

for ax in axs.flatten():
    edgeformat(ax)

df = metrics_df

hist1d(df['sc_z_334'], bins=50, range=[125,150]).plot(axs[0,0], lw=2)
axs[0,0].set_xlim(128,148)
axs[0,0].set_ylim(0,0.25)

hist1d(df['sc_z_1124'], bins=50, range=[130,155]).plot(axs[0,1], lw=2)
axs[0,1].set_xlim(128,153)
axs[0,1].set_ylim(0,0.25)

hist1d(df['sc_z_1126'], bins=50, range=[130,150]).plot_v(axs[1,2], lw=2)
axs[1,2].set_xlim(0,0.25)


hist = hist2d(metrics_df['sc_z_334'], metrics_df['sc_z_1126'], bins=40, range=[[125,150],[125,150]])
hist.dens2d_preset2(axs[1,0], lw=0.5, nlevels=10, no_last_contour_line=False, set_aspect=False)
axs[1,0].set_ylim(128,153)

hist = hist2d(metrics_df['sc_z_1124'], metrics_df['sc_z_1126'], bins=40, range=[[125,155],[125,150]])
hist.dens2d_preset2(axs[1,1], lw=0.5, nlevels=15, no_last_contour_line=False, set_aspect=False)
axs[1,1].set_ylim(128,153)


axs[1,0].set_aspect('auto')
axs[1,0].set_xlabel(r'$z$(R334-C$\zeta$) [$\mathrm{\AA}$]', fontsize=16)
axs[1,0].set_ylabel(r'$z$(E1126-C$\delta$) [$\mathrm{\AA}$]', fontsize=16)
axs[1,1].set_xlabel(r'$z$(E1124-C$\delta$) [$\mathrm{\AA}$]', fontsize=16)


axs[1,0].set_xticks(np.arange(130,145+5,5))
axs[1,0].set_xticklabels(np.arange(130,145+5,5)-130, fontsize=14)
axs[1,0].set_yticks(np.arange(130,150+5,5))
axs[1,0].set_yticklabels(np.arange(130,150+5,5)-130, fontsize=14)

axs[1,1].set_xticks(np.arange(130,150+5,5))
axs[1,1].set_xticklabels(np.arange(130,150+5,5)-130, fontsize=14)

# savefig("sidechain_sampling.pdf")

In [ ]:
# Working
percent_down_out = []
percent_e1124_out = []

# No data for the last voltage (E=+0.032)
for v in voltages[:-1]:
    trajs = get_trajid(voltage=v)
    df = metrics_df.query("traj_id in @trajs").copy()

    r334_states = (df['sc_z_334'] < 137).astype(int)
    df['r334'] = r334_states
    e1126_states = (df['sc_z_1126'] > 139).astype(int)
    df['e1126'] = e1126_states
    e1124_states = (df['sc_z_1124'] > 140).astype(int)
    df['e1124'] = e1124_states

    down_out_fraction = pd.crosstab(df['r334'], df['e1126'], normalize=True).loc[1,1]
    percent_down_out.append(down_out_fraction)
    ddf = df.query("r334 == 1 & e1126 == 1")
    e1124_out_fraction = ddf['e1124'].value_counts(normalize=True)[1]
    percent_e1124_out.append(e1124_out_fraction)
    
percent_down_bothout = np.array(percent_down_out)*np.array(percent_e1124_out)

# IV-analysis

In [ ]:
states_df['loopglu'] = -1
open_state_index = states_df.query('tmpc1v2 == 3').index.values
states_df.loc[open_state_index, 'loopglu'] = (states_df.loc[open_state_index, 'e1126'] & states_df.loc[open_state_index, 'e1124']).values

# states_df['r334i'] = states_df['r334_3d'].isin([2,4]).astype(int)
states_df

In [ ]:
iv_main = discrete_iv(traj_ids)
iv_main.prepare(states_df)
iv_main.compile_iv(breakdown_by='loopglu', breakdown_labels=[0,1], restraint_string='tmpc1v2 == 3 & r334 == 1')

fig, axs = plt.subplots(figsize=(6,6))
iv_main.ivplot_preset1(axs)
iv_main.ivplot(axs, state_labels_values={0: 'Glu in', 1: 'Glu out'}, color_dict={0: 'blue', 1: 'red'})
axs.legend(loc=0, fontsize=16)
# savefig("iv_byloop.pdf")

fig, axs = plt.subplots(figsize=(6,6))
iv_main.ivplot_preset2(axs)
iv_main.ivplot(axs, state_labels_values={0: 'loop-in', 1: 'loop-out'}, color_dict={0: 'blue', 1: 'red'}, states2fit=[0,1])
axs.set_xticks(np.linspace(-120,120,5))
axs.set_xticklabels(np.round(np.linspace(-120,120,5), 1).astype(int), fontsize=20)
axs.set_yticks(np.linspace(-1.2e-12,1.2e-12,5))
axs.set_yticklabels(np.round(np.linspace(-1.2e-12,1.2e-12,5)*1e12, 1), fontsize=20)
# savefig("iv_byloop_zoomed_inset.pdf")

In [ ]:
def quotient_error(a, b, s_a, s_b):
    return np.sqrt((s_a/a)**2 + (s_b/b)**2) * np.abs(a/b)

In [ ]:
fig, axs = plt.subplots()
edgeformat(axs)

iv_main = discrete_iv(traj_ids)
iv_main.prepare(states_df)
iv_main.compile_iv(breakdown_by='tmpc1v2', breakdown_labels=[3], restraint_string='tmpc1v2 == 3')

iv_vals = iv_main.iv_vals.query('state_label == 3')

iv_vals['g'] = iv_vals['Iz'] / iv_vals['V']
iv_vals['g_err'] = quotient_error(iv_vals['Iz'], iv_vals['V'], iv_vals['Iz_err'], iv_vals['V_err']).values

# TODO: clunky implementation
# No data for the last voltage (E=+0.032)
voltages_select = np.argwhere(voltages < 0.032).flatten()
iv_vals = iv_vals.iloc[voltages_select]


# colors = iv_vals['V']
colors = (iv_vals['V'] > 0)
axs.scatter(iv_vals['g']*1e3*1e12, 
            percent_down_bothout, c=colors, edgecolor='black', linewidth=0.5)
axs.errorbar(iv_vals['g']*1e3*1e12, percent_down_bothout, xerr=iv_vals['g_err']*1e3*1e12, lw=0.5, capsize=6, capthick=0.5, ecolor='black', fmt='none', zorder=0)

texts = []
for row, frac in zip(iv_vals.iterrows(), percent_down_bothout):
    _, row = row
    texts.append(axs.text(row['g']*1e3*1e12, frac, int(round(row['V'], -1)), fontsize=10))
adjust_text(texts)

xrange = [-2,15] 
yrange = [0,0.5]
    
axs.set_xlabel(r"$g$ [pS]", fontsize=16)
axs.set_ylabel(r"$f_{\mathrm{down-out-out}}$", fontsize=16)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)

# # Make the plot appear as square
axs.set_aspect(np.diff(xrange)[0]/np.diff(yrange)[0])
axs.grid(True, ls='--')

# savefig("conductance_vs_fraction_state.pdf")